In [3]:
from torch import nn
from d2l import torch as d2l
import torch
import math
import numpy as np
net=nn.Sequential(
    nn.Linear(20,256),
    nn.ReLU(),
    nn.Linear(256,10)
)
x=torch.normal(0,1,(2,20))
net(x)

tensor([[-0.1171,  0.0308,  0.0767, -0.1855,  0.3301, -0.0563,  0.1596, -0.1062,
         -0.2404, -0.2420],
        [-0.4693, -0.6231,  0.5358,  0.1676, -0.1126, -0.4171,  0.0014,  0.1881,
         -0.3684,  0.2428]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):#自定义神经网络
    def __init__(self):
        super().__init__()
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
    def forward(self,x):
        return self.out(nn.functional.relu(self.hidden(x)))

In [5]:
net=MLP()
net(x)

tensor([[ 0.0066,  0.0689,  0.1696,  0.2444,  0.2234, -0.1661,  0.2629, -0.0161,
          0.1622,  0.1526],
        [ 0.3730,  0.1781,  0.4780, -0.0409,  0.3053,  0.0421,  0.3990,  0.1468,
         -0.1722,  0.6549]], grad_fn=<AddmmBackward0>)

#### enumerate()函数主要是将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，形成一组键值对

In [53]:
class MYS(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx,module in enumerate(args):
            self._modules[str(idx)]=module
    def forward(self, x):
        for b in self._modules.values():#加两根下划线的话，无法从外部访问self.modules
            x=b(x)
        return x

In [56]:
net=MYS(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(x)

tensor([[-0.0195,  0.2699, -0.2267, -0.0524,  0.0661,  0.1545,  0.0299,  0.1127,
          0.2180, -0.3290],
        [ 0.3626,  0.3333,  0.2494,  0.4467, -0.0353,  0.2803,  0.1796,  0.3860,
          0.6106, -0.3706]], grad_fn=<AddmmBackward0>)

### 实现$y=cW^TX$

In [69]:
#input 20,output 10,hidden 256
class fixedhiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.c=torch.normal(0,10,(256,256),requires_grad=False)#常数隐藏层，但是不梯度更新参数，所以requires-grad=False
        self.layer1=nn.Linear(20,256)
        self.layer2=nn.Linear(256,10)
    def forward(self,x):
        x=self.layer1(x)
        x=nn.functional.relu(torch.matmul(x,self.c))
        x=self.layer2(x)
        #做一个控制流
        while x.abs().sum()>1:
            x/=2
        return x.sum()

In [86]:
net=fixedhiddenMLP()
net(x)

tensor(-0.1862, grad_fn=<SumBackward0>)

### 内部镶嵌各种组合块

In [91]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(20,256),
            nn.ReLU(),
            nn.Linear(256,256),
            nn.ReLU()
        )
        self.layer=nn.Linear(256,10)
    def forward(self,x):
        return self.layer(self.net(x))

### 外部随意组合

In [92]:
nett=nn.Sequential(MLP(),nn.Linear(10,20),fixedhiddenMLP())
nett(x)

tensor(0.1540, grad_fn=<SumBackward0>)

### 关于效率问题，我们担心速度极快的GPU可能要等到CPU运行Python代码后才能运行另一个作业。